In [6]:
using Pkg
# Run once if needed:
# Pkg.add(["QuantumToolbox", "QuantumOptics", "Plots", "BenchmarkTools"])

using QuantumToolbox
using QuantumOptics
using Plots
using BenchmarkTools

# Short aliases to avoid long names and fix ambiguity
const QT = QuantumToolbox
const QO = QuantumOptics

println("Packages loaded.")


Packages loaded.


In [7]:
## QuantumToolbox.jl implementation of 2-site JCH

"""
    build_jch_2site_qt(N, ωc, ωa, g, J)

Construct the 2-site Jaynes–Cummings–Hubbard Hamiltonian
with QuantumToolbox.jl.

Order of subsystems in the tensor product:
    (Cavity A, Atom A, Cavity B, Atom B)

Hilbert space dims = [N, 2, N, 2].
Returns:
    H, ops (NamedTuple with a1, a2, sm1, sm2, sz1, sz2)
"""
function build_jch_2site_qt(N::Int, ωc::Real, ωa::Real, g::Real, J::Real)
    # System A
    a1  = QT.tensor(QT.destroy(N), QT.qeye(2),
                    QT.qeye(N),    QT.qeye(2))
    sm1 = QT.tensor(QT.qeye(N),    QT.sigmam(),
                    QT.qeye(N),    QT.qeye(2))
    sz1 = QT.tensor(QT.qeye(N),    QT.sigmaz(),
                    QT.qeye(N),    QT.qeye(2))

    # System B
    a2  = QT.tensor(QT.qeye(N), QT.qeye(2),
                    QT.destroy(N), QT.qeye(2))
    sm2 = QT.tensor(QT.qeye(N), QT.qeye(2),
                    QT.qeye(N), QT.sigmam())
    sz2 = QT.tensor(QT.qeye(N), QT.qeye(2),
                    QT.qeye(N), QT.sigmaz())

    # Bare cavity energies
    Hc   = ωc * (a1' * a1 + a2' * a2)

    # Bare atomic energies
    Ha   = 0.5 * ωa * (sz1 + sz2)

    # JC light–matter interaction (both sites)
    Hint = g * (a1' * sm1 + a1 * sm1' +
                a2' * sm2 + a2 * sm2')

    # Photon hopping between cavities
    Hhop = J * (a1' * a2 + a2' * a1)

    H = Hc + Ha + Hint + Hhop

    return H, (
        a1  = a1,  a2  = a2,
        sm1 = sm1, sm2 = sm2,
        sz1 = sz1, sz2 = sz2,
    )
end

"""
    run_jch_2site_qt(; N, ωc, ωa, g, J, tmax, nt)

Run closed-system unitary dynamics for the 2-site JCH model
using QuantumToolbox.sesolve.

Returns:
    tlist, ex_qt

where ex_qt is a NamedTuple:
    ex_qt.nA  = ⟨σ⁺₁σ⁻₁⟩  (atom A excited population)
    ex_qt.nB  = ⟨σ⁺₂σ⁻₂⟩  (atom B excited population)
    ex_qt.n1  = ⟨a₁† a₁⟩  (cavity A photon number)
    ex_qt.n2  = ⟨a₂† a₂⟩  (cavity B photon number)
"""
function run_jch_2site_qt(; N::Int = 4,
                            ωc::Real = 1.0,
                            ωa::Real = 1.0,
                            g::Real  = 0.1,
                            J::Real  = 0.05,
                            tmax::Real = 50.0,
                            nt::Int = 200)

    H, ops = build_jch_2site_qt(N, ωc, ωa, g, J)

    # Initial state: atom A excited, atom B ground, both cavities empty
    ψ0 = QT.tensor(
        QT.basis(N, 0),  # cavity A: |0⟩
        QT.basis(2, 0),  # atom A:   |e⟩
        QT.basis(N, 0),  # cavity B: |0⟩
        QT.basis(2, 1),  # atom B:   |g⟩
    )

    # Observables
    nA = ops.sm1' * ops.sm1      # atom A excitation
    nB = ops.sm2' * ops.sm2      # atom B excitation
    n1 = ops.a1'  * ops.a1       # cavity A photons
    n2 = ops.a2'  * ops.a2       # cavity B photons

    tlist = range(0.0, tmax; length = nt)

    prob = QT.sesolveProblem(H, ψ0, tlist;
                             e_ops = [nA, nB, n1, n2],
                             progress_bar = false)
    sol = QT.sesolve(prob)

    ex_qt = (
        nA = sol.expect[1],
        nB = sol.expect[2],
        n1 = sol.expect[3],
        n2 = sol.expect[4],
    )

    return tlist, ex_qt
end

println("QuantumToolbox JCH functions defined.")


QuantumToolbox JCH functions defined.


In [8]:
## QuantumOptics.jl implementation of 2-site JCH

"""
    build_jch_2site_qo(N, ωc, ωa, g, J)

Construct the 2-site Jaynes–Cummings–Hubbard Hamiltonian
with QuantumOptics.jl.

Returns:
    H, ops, bases

where
    ops   is a NamedTuple with a1, a2, sm1, sm2, sz1, sz2
    bases is a NamedTuple with b_c (FockBasis) and b_a (SpinBasis)
"""
function build_jch_2site_qo(N::Int, ωc::Real, ωa::Real, g::Real, J::Real)
    # Local bases
    b_c = QO.FockBasis(N)
    b_a = QO.SpinBasis(1//2)

    # System A
    a1  = QO.tensor(QO.destroy(b_c), QO.identityoperator(b_a),
                    QO.identityoperator(b_c), QO.identityoperator(b_a))
    sm1 = QO.tensor(QO.identityoperator(b_c), QO.sigmam(b_a),
                    QO.identityoperator(b_c), QO.identityoperator(b_a))
    sz1 = QO.tensor(QO.identityoperator(b_c), QO.sigmaz(b_a),
                    QO.identityoperator(b_c), QO.identityoperator(b_a))

    # System B
    a2  = QO.tensor(QO.identityoperator(b_c), QO.identityoperator(b_a),
                    QO.destroy(b_c), QO.identityoperator(b_a))
    sm2 = QO.tensor(QO.identityoperator(b_c), QO.identityoperator(b_a),
                    QO.identityoperator(b_c), QO.sigmam(b_a))
    sz2 = QO.tensor(QO.identityoperator(b_c), QO.identityoperator(b_a),
                    QO.identityoperator(b_c), QO.sigmaz(b_a))

    # Hamiltonian terms
    Hc   = ωc * (QO.dagger(a1) * a1 + QO.dagger(a2) * a2)
    Ha   = 0.5 * ωa * (sz1 + sz2)
    Hint = g * (QO.dagger(a1)*sm1 + a1*QO.dagger(sm1) +
                QO.dagger(a2)*sm2 + a2*QO.dagger(sm2))
    Hhop = J * (QO.dagger(a1)*a2 + QO.dagger(a2)*a1)

    H = Hc + Ha + Hint + Hhop

    ops = (a1=a1, a2=a2, sm1=sm1, sm2=sm2, sz1=sz1, sz2=sz2)
    bases = (b_c=b_c, b_a=b_a)

    return H, ops, bases
end

"""
    run_jch_2site_qo(; N, ωc, ωa, g, J, tmax, nt)

Run closed-system unitary dynamics for the 2-site JCH model
using QuantumOptics.timeevolution.schroedinger.

Returns:
    tlist, ex_qo

where ex_qo is a NamedTuple matching ex_qt.
"""
function run_jch_2site_qo(; N::Int = 4,
                            ωc::Real = 1.0,
                            ωa::Real = 1.0,
                            g::Real  = 0.1,
                            J::Real  = 0.05,
                            tmax::Real = 50.0,
                            nt::Int = 200)

    H, ops, bases = build_jch_2site_qo(N, ωc, ωa, g, J)
    b_c, b_a = bases.b_c, bases.b_a

    # Initial state: atom A excited, atom B ground, both cavities empty
    ψ0 = QO.tensor(
        QO.fockstate(b_c, 0),   # cavity A |0⟩
        QO.spinup(b_a),         # atom A   |e⟩
        QO.fockstate(b_c, 0),   # cavity B |0⟩
        QO.spindown(b_a),       # atom B   |g⟩
    )

    # Observables
    nA = QO.dagger(ops.sm1) * ops.sm1
    nB = QO.dagger(ops.sm2) * ops.sm2
    n1 = QO.dagger(ops.a1)  * ops.a1
    n2 = QO.dagger(ops.a2)  * ops.a2

    tlist = collect(range(0.0, tmax; length = nt))

    tout, ψt = QO.timeevolution.schroedinger(tlist, ψ0, H)

    ex_qo = (
        nA = QO.expect(nA, ψt),
        nB = QO.expect(nB, ψt),
        n1 = QO.expect(n1, ψt),
        n2 = QO.expect(n2, ψt),
    )

    return tlist, ex_qo
end

println("QuantumOptics JCH functions defined.")


QuantumOptics JCH functions defined.


In [9]:
# Common parameters
N   = 4
ωc  = 1.0
ωa  = 1.0
g   = 0.1
J   = 0.05
tmax = 50.0
nt   = 400

println("Running QuantumToolbox.jl...")
t_qt, ex_qt = run_jch_2site_qt(N=N, ωc=ωc, ωa=ωa, g=g, J=J, tmax=tmax, nt=nt)

println("Running QuantumOptics.jl...")
t_qo, ex_qo = run_jch_2site_qo(N=N, ωc=ωc, ωa=ωa, g=g, J=J, tmax=tmax, nt=nt)

println("Time grids equal? ", all(isapprox.(t_qt, t_qo)))


Running QuantumToolbox.jl...
Running QuantumOptics.jl...
Time grids equal? true


In [11]:
maximum(abs, imag.(ex_qt.nA)), maximum(abs, imag.(ex_qo.nA))


(0.0, 0.0)

In [12]:
## Plot comparisons (using real parts of expectation values)

default(titlefont=12, legendfontsize=8, guidefont=10)

# Take real parts to avoid ComplexF64 plotting issue
nA_qt = real.(ex_qt.nA)
nB_qt = real.(ex_qt.nB)
n1_qt = real.(ex_qt.n1)
n2_qt = real.(ex_qt.n2)

nA_qo = real.(ex_qo.nA)
nB_qo = real.(ex_qo.nB)
n1_qo = real.(ex_qo.n1)
n2_qo = real.(ex_qo.n2)

# Atom excitations
p1 = plot(t_qt, nA_qt, label="Atom A (QT)", xlabel="t", ylabel="⟨σ⁺σ⁻⟩", lw=2)
plot!(p1, t_qo, nA_qo, label="Atom A (QO)", ls=:dash, lw=2)
plot!(p1, t_qt, nB_qt, label="Atom B (QT)", lw=2)
plot!(p1, t_qo, nB_qo, label="Atom B (QO)", ls=:dash, lw=2)
title!(p1, "Atomic Excitations – QuantumToolbox vs QuantumOptics")

# Cavity photon numbers
p2 = plot(t_qt, n1_qt, label="Cavity 1 (QT)", xlabel="t", ylabel="⟨a†a⟩", lw=2)
plot!(p2, t_qo, n1_qo, label="Cavity 1 (QO)", ls=:dash, lw=2)
plot!(p2, t_qt, n2_qt, label="Cavity 2 (QT)", lw=2)
plot!(p2, t_qo, n2_qo, label="Cavity 2 (QO)", ls=:dash, lw=2)
title!(p2, "Cavity Photon Numbers – QuantumToolbox vs QuantumOptics")

plot(p1, p2, layout=(2,1), size=(800,600))


LoadError: Cannot convert Float64 to series data for plotting

In [15]:
import Pkg; Pkg.add("time")

LoadError: The following package names could not be resolved:
 * time (not found in project, manifest or registry)
[36m   Suggestions:[39m [0m[1mT[22m[0m[1mi[22m[0m[1mm[22m[0m[1me[22mrs [0m[1mT[22m[0m[1mi[22m[0m[1mm[22m[0m[1me[22mDag [0m[1mT[22m[0m[1mi[22m[0m[1mm[22m[0m[1me[22mAxes [0m[1mT[22m[0m[1mi[22m[0m[1mm[22m[0m[1me[22mSpans [0m[1mT[22m[0m[1mi[22m[0m[1mm[22m[0m[1me[22mZones [0m[1mT[22m[0m[1mi[22m[0m[1mm[22m[0m[1me[22mArrays

In [17]:
import time
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
from julia import Main

# ==========================================
# 1. SETUP JULIA ENGINES
# ==========================================

print("Initializing Julia engines...")

# --- CORRECTED JULIA CODE (No 'using time' here!) ---
julia_code = """
import Pkg
# Uncomment if packages are missing:
# Pkg.add("QuantumOptics")
# Pkg.add("QuantumToolbox")

using QuantumOptics
using QuantumToolbox

# --- Engine 1: QuantumOptics.jl ---
function run_optics_bench(N)
    # Define Basis
    b_cav = FockBasis(N)
    b_atom = SpinBasis(1//2)
    b_site = b_cav ⊗ b_atom
    
    # Define Operators
    a = destroy(b_cav) ⊗ one(b_atom)
    sm = one(b_cav) ⊗ sigmam(b_atom)
    sz = one(b_cav) ⊗ sigmaz(b_atom)
    
    # Expand to 2 sites (Tensor Product)
    a1 = a ⊗ one(b_site); sm1 = sm ⊗ one(b_site); sz1 = sz ⊗ one(b_site)
    a2 = one(b_site) ⊗ a; sm2 = sm ⊗ one(b_site); sz2 = sz ⊗ one(b_site)
    
    # Hamiltonian
    H = 1.0*a1'*a1 + 0.5*sz1 + 0.05*(a1'*sm1 + a1*sm1') +
        1.0*a2'*a2 + 0.5*sz2 + 0.05*(a2'*sm2 + a2*sm2') - 
        0.05*(a1'*a2 + a1*a2')
        
    # Initial State
    psi0 = fockstate(b_cav, 0) ⊗ spinup(b_atom) ⊗ fockstate(b_cav, 0) ⊗ spindown(b_atom)
    T = [0:0.5:50;]
    
    # Run Evolution
    timeevolution.schroedinger(T, psi0, H)
end

# --- Engine 2: QuantumToolbox.jl ---
function run_toolbox_bench(N)
    # Define Operators (QuTiP Syntax)
    a = destroy(N)
    sz = sigmaz()
    sm = sigmam()
    Ic = qeye(N)
    Ia = qeye(2)
    
    # Tensor Products
    a1  = tensor(a, Ia, Ic, Ia); sm1 = tensor(Ic, sm, Ic, Ia); sz1 = tensor(Ic, sz, Ic, Ia)
    a2  = tensor(Ic, Ia, a, Ia); sm2 = tensor(Ic, Ia, Ic, sm); sz2 = tensor(Ic, Ia, Ic, sz)
    
    # Hamiltonian
    H = 1.0*a1'*a1 + 0.5*sz1 + 0.05*(a1'*sm1 + a1*sm1') +
        1.0*a2'*a2 + 0.5*sz2 + 0.05*(a2'*sm2 + a2*sm2') - 
        0.05*(a1'*a2 + a1*a2')
        
    # Initial State
    psi0 = tensor(basis(N,0), basis(2,0), basis(N,0), basis(2,1))
    tlist = range(0, 50, length=100)
    
    # Run Evolution
    mesolve(H, psi0, tlist, c_ops=[], e_ops=[])
end
"""

# Compile Julia Code
Main.eval(julia_code)

# Warmup to clear JIT compilation
print("Warming up Julia JIT compiler...")
Main.run_optics_bench(2)
Main.run_toolbox_bench(2)
print("Engines Ready!")

# ==========================================
# 2. RUN BENCHMARK LOOP
# ==========================================

def run_all_benchmarks(N_values):
    times_qutip = []
    times_optics = []
    times_toolbox = []
    
    print(f"{'N':<5} | {'QuTiP (s)':<12} | {'Optics (s)':<12} | {'Toolbox (s)':<12}")
    print("-" * 55)
    
    for N in N_values:
        # --- A. QuTiP (Python) ---
        start = time.time()
        # Re-build system to include construction time overhead
        a = tensor(destroy(N), qeye(2))
        sm = tensor(qeye(N), sigmam())
        sz = tensor(qeye(N), sigmaz())
        a1 = tensor(a, qeye(N), qeye(2)); sm1 = tensor(sm, qeye(N), qeye(2)); sz1 = tensor(sz, qeye(N), qeye(2))
        a2 = tensor(qeye(N), qeye(2), a); sm2 = tensor(qeye(N), qeye(2), sm); sz2 = tensor(qeye(N), qeye(2), sz)
        
        H = 1.0*a1.dag()*a1 + 0.5*sz1 + 0.05*(a1.dag()*sm1 + a1*sm1.dag()) + \
            1.0*a2.dag()*a2 + 0.5*sz2 + 0.05*(a2.dag()*sm2 + a2*sm2.dag()) - \
            0.05*(a1.dag()*a2 + a1*a2.dag())
            
        psi0 = tensor(basis(N,0), basis(2,0), basis(N,0), basis(2,1))
        tlist = np.linspace(0, 50, 100)
        
        mesolve(H, psi0, tlist, [], [])
        t_qutip = time.time() - start
        
        # --- B. QuantumOptics.jl ---
        start = time.time()
        Main.run_optics_bench(N)
        t_optics = time.time() - start
        
        # --- C. QuantumToolbox.jl ---
        start = time.time()
        Main.run_toolbox_bench(N)
        t_toolbox = time.time() - start
        
        # Store & Print
        times_qutip.append(t_qutip)
        times_optics.append(t_optics)
        times_toolbox.append(t_toolbox)
        
        print(f"{N:<5} | {t_qutip:<12.4f} | {t_optics:<12.4f} | {t_toolbox:<12.4f}")
        
    return times_qutip, times_optics, times_toolbox

# Run for N = 2, 4, 6, 8, 10, 12
N_list = [2, 4, 6, 8, 10, 12]
t_qt, t_qo, t_tb = run_all_benchmarks(N_list)

# ==========================================
# 3. PLOT RESULTS
# ==========================================
plt.figure(figsize=(10, 6))
plt.plot(N_list, t_qt, 'o-', label='QuTiP (Python)', color='blue', linewidth=2)
plt.plot(N_list, t_qo, 's-', label='QuantumOptics.jl', color='red', linewidth=2)
plt.plot(N_list, t_tb, '^-', label='QuantumToolbox.jl', color='green', linewidth=2)

plt.yscale('log')
plt.xlabel('Hilbert Space Cutoff (N)', fontsize=12)
plt.ylabel('Execution Time (seconds)', fontsize=12)
plt.title('Performance Benchmark: 2-Site JCH Model', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, which="both", ls="--", alpha=0.6)
plt.tight_layout()
plt.show()

LoadError: ArgumentError: Package time not found in current path.
- Run `import Pkg; Pkg.add("time")` to install the time package.